In [ ]:
# ================= Feature Store Setup =================
import sqlite3
import pandas as pd
from datetime import datetime
from IPython.display import display

# Connect to SQLite database (will create if not exists)
conn = sqlite3.connect("feature_store.db")
cursor = conn.cursor()

# -------- Step 1: Create metadata table --------
cursor.execute('''
    CREATE TABLE IF NOT EXISTS feature_metadata (
        feature_name TEXT PRIMARY KEY,
        description TEXT,
        source TEXT,
        version TEXT,
        created_at TEXT
    )
''')

# -------- Step 2: Create feature store table --------
cursor.execute('''
    CREATE TABLE IF NOT EXISTS feature_store (
        customerID TEXT PRIMARY KEY,
        tenure INTEGER,
        MonthlyCharges REAL,
        TotalCharges REAL,
        Contract_OneYear INTEGER,
        Contract_TwoYear INTEGER,
        PaymentMethod_CreditCard INTEGER,
        PaymentMethod_ElectronicCheck INTEGER,
        PaymentMethod_MailedCheck INTEGER,
        Churn INTEGER
    )
''')

# -------- Step 3: Insert metadata for features --------
features_meta = [
    ("tenure", "Number of months the customer has stayed", "Telco DB", "v1", str(datetime.now())),
    ("MonthlyCharges", "Monthly fee charged to customer", "Telco DB", "v1", str(datetime.now())),
    ("TotalCharges", "Total amount charged", "Telco DB", "v1", str(datetime.now())),
    ("Contract_OneYear", "Contract type is one year", "Engineered", "v1", str(datetime.now())),
    ("Contract_TwoYear", "Contract type is two year", "Engineered", "v1", str(datetime.now())),
    ("PaymentMethod_CreditCard", "Payment via credit card", "Engineered", "v1", str(datetime.now())),
    ("PaymentMethod_ElectronicCheck", "Payment via electronic check", "Engineered", "v1", str(datetime.now())),
    ("PaymentMethod_MailedCheck", "Payment via mailed check", "Engineered", "v1", str(datetime.now())),
    ("Churn", "Whether customer churned (0/1)", "Label", "v1", str(datetime.now())),
    ("Churn", "Whether buyer churned (0/1)", "Label", "v1", str(datetime.now())),
]

cursor.executemany('''
    INSERT OR IGNORE INTO feature_metadata 
    (feature_name, description, source, version, created_at)
    VALUES (?, ?, ?, ?, ?)
''', features_meta)

# -------- Step 4: Insert example customer features --------
cursor.executemany('''
    INSERT OR REPLACE INTO feature_store VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', [
    ("C12345", 24, 55.2, 1324.5, 1, 0, 1, 0, 0, 0),
    ("C12346", 12, 70.0, 840.0, 0, 1, 0, 1, 0, 1),
    ("C12347", 36, 65.0, 2340.0, 0, 1, 0, 0, 1, 1),
    ("C12348", 6, 50.0, 300.0, 1, 0, 1, 0, 0, 0),
    ("C12349", 18, 60.0, 1080.0, 1, 0, 0, 1, 0, 1)
])


conn.commit()

# -------- Step 5: Retrieve features for training/inference --------
features = pd.read_sql("SELECT * FROM feature_store", conn)
print("Feature Store Data:")
display(features)

# -------- Step 6: Retrieve metadata for documentation --------
metadata = pd.read_sql("SELECT * FROM feature_metadata", conn)
print(" Feature Metadata:")
display(metadata)

# Close connection
conn.close()
